# Sprint 機械学習スクラッチ SVM

class ScratchSVMClassifier():
    """
    SVM分類器のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    kernel : str
      カーネルの種類。線形カーネル（linear）か多項式カーネル（polly）
    threshold : float
      サポートベクターを選ぶための閾値
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.n_support_vectors : int
      サポートベクターの数
    self.index_support_vectors : 次の形のndarray, shape (n_support_vectors,)
      サポートベクターのインデックス
    self.X_sv :  次の形のndarray, shape(n_support_vectors, n_features)
      サポートベクターの特徴量
    self.lam_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの未定乗数
    self.y_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターのラベル

    """
    def __init__(self, num_iter, lr, kernel='linear', threshold=1e-5, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose
    def fit(self, X, y, X_val=None, y_val=None):
        """
        SVM分類器を学習する。検証データが入力された場合はそれに対する精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
        pass
    def predict(self, X):
        """
        SVM分類器を使いラベルを推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            SVM分類器による推定結果
        """
        pass
        return

In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

## 【問題1】ラグランジュの未定乗数法による最急降下
SVMの学習は、ラグランジュの未定乗数法を用います。サンプル数分のラグランジュ乗数 $\lambda$ を用意して、以下の式により更新していきます。この計算を行うメソッドをScratchSVMClassifierクラスに実装してください。

$$
λ
n
e
w
i
=
λ
i
+
α
(
1
−
n
∑
j
=
1
-
λ
j
y
i
y
j
k
(
x
i
,
x
j
)
)
$$

ここで $k(x_i, x_j)$ はカーネル関数です。線形カーネルの場合は次のようになります。他のカーネル関数にも対応できるように、この部分は独立したメソッドとしておきましょう。

$$
k
(
x
i
,
x
j
)
=
x
T
i
x
j
$$

条件として、更新毎に $\lambda_i >= 0$を満たす必要があります。満たさない場合は $\lambda_i = 0$とします。


$i, j$ : サンプルのインデックス


$\lambda_i^{new}$ : 更新後のi番目のサンプルのラグランジュ乗数


$\lambda_i$ : 更新前のi番目のサンプルのラグランジュ乗数


$\alpha$ : 学習率


$\lambda_j$ : j番目のサンプルのラグランジュ乗数


$y_i$ : i番目のサンプルのラベル


$y_j$ : j番目のサンプルのラベル


$x_i$ : i番目のサンプルの特徴量ベクトル


$x_j$ : j番目のサンプルの特徴量ベクトル


あるサンプルに対しての全てのサンプルとの関係を計算していくことになります。



In [122]:
def karnel_func(X):
    return np.dot(X, X.T)

In [277]:
# テスト用
X_tes = np.array([[1, 2], [3, 4], [5, 6]])
y_tes = np.array([1, 1, -1])

In [187]:
def lagrange_func(X, y, alpha=0.01):    
    lamda_i_arr = np.ones(X.shape[0])
    lamda_j = np.full(X.shape[0], 2)
    
    y = y.reshape(-1, 1)
    y_matrix = np.dot(y, y.T)
    print(y_matrix)
    
    kanel = karnel_func(X)
    print(kanel)
    
    ky = y_matrix*kanel
    print(ky)
        
    kg = alpha*(1- np.dot((y_matrix*karnel_func(X)), lamda_j))
    print(kg, "111")
    
    for i in range(X.shape[0]):
        #print(lamda_i_arr[i])
        #print(lamda_i_arr)
        #print(lamda_j)
        lamda_i_arr[i] = lamda_i_arr[i] + kg[i]
    
    print(lamda_i_arr)
    print(lamda_i_arr.shape)
    #if lamda_i_arr < 0:
        #lamda_i = 0

    return lamda_i_arr

In [188]:
lagrange_func(X_tes, y_tes)

[[1 1 0]
 [1 1 0]
 [0 0 0]]
[[ 5 11 17]
 [11 25 39]
 [17 39 61]]
[[ 5 11  0]
 [11 25  0]
 [ 0  0  0]]
[-0.31 -0.71  0.01] 111
[0.69 0.29 1.01]
(3,)


array([0.69, 0.29, 1.01])

In [199]:
np.where(aaa < 0, 0, aaa)

array([0.69, 0.29, 1.01])

In [209]:
bbb = np.array([-1, -0.79, 0.0000002 ,1 , 3, 100])

In [210]:
np.where(bbb > 1e-5)

(array([3, 4, 5]),)

In [202]:
np.argmax(aaa)

2

In [203]:
np.max(aaa)

1.01

In [204]:
X_tes[np.argmax(aaa)]

array([5, 6])

In [241]:
len(aaa)

3

In [ ]:
aaa.

In [301]:
class ScratchSVMClassifier():
    """
    SVM分類器のスクラッチ実装

    Parameters
    ----------
    num_iter : int
      イテレーション数
    lr : float
      学習率
    kernel : str
      カーネルの種類。線形カーネル（linear）か多項式カーネル（polly）
    threshold : float
      サポートベクターを選ぶための閾値
    verbose : bool
      学習過程を出力する場合はTrue

    Attributes
    ----------
    self.n_support_vectors : int
      サポートベクターの数
    self.index_support_vectors : 次の形のndarray, shape (n_support_vectors,)
      サポートベクターのインデックス
    self.X_sv :  次の形のndarray, shape(n_support_vectors, n_features)
      サポートベクターの特徴量
    self.lam_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターの未定乗数
    self.y_sv :  次の形のndarray, shape(n_support_vectors, 1)
      サポートベクターのラベル

    """
    def __init__(self, num_iter, lr, kernel='linear', threshold=1e-5, verbose=False):
        # ハイパーパラメータを属性として記録
        self.iter = num_iter
        self.lr = lr
        self.kernel = kernel
        self.threshold = threshold
        self.verbose = verbose
    def fit(self, X, y, X_val=None, y_val=None):
        """
        SVM分類器を学習する。検証データが入力された場合はそれに対する精度もイテレーションごとに計算する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        X_val : 次の形のndarray, shape (n_samples, n_features)
            検証データの特徴量
        y_val : 次の形のndarray, shape (n_samples, )
            検証データの正解値
        """
        np.random.seed(0)
        self.lamda_i_arr = np.random.random(X.shape[0])
        self.lamda_j = np.random.rand(X.shape[0])
        
        print(self.lamda_i_arr)
        print(self.lamda_j)
        
        self.X, self.y, self.X_val, self.y_val = self._setting(X, y, X_val, y_val)       
        
        #self.n_support_vectors = 3
        #self.lam_sv = self._lagrange_func(self.X, self.y)  #サポートベクターの未定乗数
        #self.index_support_vectors = self.X[np.argmax(self.lam_sv)]  #サポートベクターのインデックス
        #self.X_sv = np.max(self.lam_sv)  #サポートベクターの特徴量
        #self.y_sv = self.y[np.argmax(self.lam_sv)]  #サポートベクターのラベル
        
        for i in range(self.iter):
            self.lam_sv = self._lagrange_func(self.X, self.y)   # ラムダ更新
            print("置換後", self.lam_sv)
            self.index_support_vectors = np.where(self.lam_sv > self.threshold)[0] #サポートベクターのインデックス
            #print(self.index_support_vectors)
            #print(self.index_support_vectors.shape)
            print("[iter{}]サポートベクターの数:{}".format(i, len(self.index_support_vectors)))
            
            self.X_sv = self.X[self.index_support_vectors]  #サポートベクターの特徴量
            #print(self.X_sv)
            self.y_sv = self.y[self.index_support_vectors]  #サポートベクターのラベル
            #print(self.y_sv)

        if self.verbose:
            #verboseをTrueにした際は学習過程を出力
            print()
                           
    def _setting(self, X, y, X_val, y_val):                   
        X_copy = np.copy(X)
        y_copy = np.copy(y)
        
        if X_val is not None and y_val is not None:
            X_val_copy = np.copy(X_val)
            y_val_copy = np.copy(y_val)
        else:
            X_val_copy = X_val
            y_val_copy = y_val

        return X_copy, y_copy, X_val_copy, y_val_copy

    def predict(self, X):
        """
        SVM分類器を使いラベルを推定する。

        Parameters
        ----------
        X : 次の形のndarray, shape (n_samples, n_features)
            サンプル

        Returns
        -------
            次の形のndarray, shape (n_samples, 1)
            SVM分類器による推定結果
        """
        karnel_Xsv = self._karnel_func(X, self.X_sv)
        print(karnel_Xsv)
        print(karnel_Xsv.shape)
        # karnel_ysv = self._karnel_func(y, self.y_sv)
        
        print(self.y_sv)
        
        print(self.y_sv.shape)
        
        pred = np.dot((self.y_sv.reshape(-1, 1)*karnel_Xsv), self.lam_sv)
        
        return pred
    
    # 問題1
    # カーネル関数
    def _karnel_func(self, array1, array2=None):
        """
        カーネル関数
        
        Parameters
        ------------
        array：行列の
        
        Returns
        
        """
        if array2 is not None:
            return np.dot(array1, array2.T)
        else:
            return np.dot(array1, array1.T)

    # 問題2
    # ラグランジュ関数    
    def _lagrange_func(self, X, y):
        """
        ラグランジュの未定乗数法
        
        Parameters
        ------------
        X : 次の形のndarray, shape (n_samples, n_features)
            訓練データの特徴量
        y : 次の形のndarray, shape (n_samples, )
            訓練データの正解値
        
        Returns        
        """        
        if X.ndim == 1:
            X.reshape(-1, 1)
        
        if y.ndim == 1:
            y = y.reshape(-1, 1)
        
        karnel_X = self._karnel_func(X)
        karnel_y = self._karnel_func(y)
        
        sigma = self.lr*(1- np.dot((karnel_y*karnel_X), self.lamda_j))
        
        for i in range(X.shape[0]):
            self.lamda_i_arr[i] = self.lamda_i_arr[i] + sigma[i]
        
        return np.where(self.lamda_i_arr < 0, 0, self.lamda_i_arr) #0以下の値を0に置換

In [302]:
# 自作テストデータで動作確認
model1 = ScratchSVMClassifier(num_iter=100, 
                              lr= 0.01,
                              kernel='linear', 
                              threshold=1e-5, 
                              verbose=False)

In [303]:
model1.fit(X_tes, y_tes)

[0.5488135  0.71518937 0.60276338]
[0.54488318 0.4236548  0.64589411]
置換後 [0.59476932 0.81123722 0.47662348]
[iter0]サポートベクターの数:3
置換後 [0.64072513 0.90728507 0.35048358]
[iter1]サポートベクターの数:3
置換後 [0.68668094 1.00333293 0.22434369]
[iter2]サポートベクターの数:3
置換後 [0.73263675 1.09938078 0.09820379]
[iter3]サポートベクターの数:3
置換後 [0.77859256 1.19542864 0.        ]
[iter4]サポートベクターの数:2
置換後 [0.82454838 1.29147649 0.        ]
[iter5]サポートベクターの数:2
置換後 [0.87050419 1.38752435 0.        ]
[iter6]サポートベクターの数:2
置換後 [0.91646   1.4835722 0.       ]
[iter7]サポートベクターの数:2
置換後 [0.96241581 1.57962005 0.        ]
[iter8]サポートベクターの数:2
置換後 [1.00837163 1.67566791 0.        ]
[iter9]サポートベクターの数:2
置換後 [1.05432744 1.77171576 0.        ]
[iter10]サポートベクターの数:2
置換後 [1.10028325 1.86776362 0.        ]
[iter11]サポートベクターの数:2
置換後 [1.14623906 1.96381147 0.        ]
[iter12]サポートベクターの数:2
置換後 [1.19219487 2.05985932 0.        ]
[iter13]サポートベクターの数:2
置換後 [1.23815069 2.15590718 0.        ]
[iter14]サポートベクターの数:2
置換後 [1.2841065  2.25195503 0.        ]
[ite

In [304]:
model1.predict(X_tes)

[[ 5 11]
 [11 25]
 [17 39]]
(3, 2)
[1 1]
(2,)


ValueError: operands could not be broadcast together with shapes (2,1) (3,2) 

In [282]:
np.random.seed(seed=0)
n_samples = 500
f0 = [-1, 2]
f1 = [2, -1]
cov = [[1.0,0.8], [0.8, 1.0]]
f0 = np.random.multivariate_normal(f0, cov, int(n_samples/2))
f1 = np.random.multivariate_normal(f1, cov, int(n_samples/2))
X = np.concatenate((f0, f1))
y = np.concatenate((np.ones((int(n_samples/2))), np.ones((int(n_samples/2))) *(-1))).astype(np.int)
random_index = np.random.permutation(np.arange(n_samples))
X = X[random_index]
y = y[random_index]

In [283]:
X

array([[ 7.72382751e-01, -2.29167329e+00],
       [-5.93349449e-01,  1.66788336e+00],
       [-2.07648560e+00,  4.87468451e-01],
       [ 1.19226877e-01,  3.62537974e+00],
       [-3.13000578e+00, -1.56731551e-01],
       [-1.78109832e+00,  1.22224904e+00],
       [ 3.99770982e+00,  1.25164011e+00],
       [ 2.15604470e+00, -3.85824429e-01],
       [ 1.94741552e+00, -1.29638961e+00],
       [ 1.58757396e+00, -1.85989193e+00],
       [ 2.05369045e+00, -9.47185530e-01],
       [-1.97439392e+00,  1.22718715e+00],
       [-3.47487306e+00,  3.70421433e-01],
       [ 1.68094977e+00, -6.36507554e-01],
       [-9.51997101e-01,  1.41989638e+00],
       [-1.23054341e+00,  2.48848983e+00],
       [-6.96789478e-01,  1.88359001e+00],
       [ 1.03842491e+00, -8.88815671e-01],
       [ 3.68706491e+00,  4.30242556e-01],
       [-1.03002856e+00,  1.27865865e+00],
       [-1.26658152e+00,  1.97258945e+00],
       [-1.97638843e-02,  2.54412654e+00],
       [ 8.60592217e-01, -2.46186096e+00],
       [ 1.

In [284]:
X.shape

(500, 2)

In [286]:
X2_train, X2_test, y2_train, y2_test =  train_test_split(X, y, test_size=0.25, random_state=0)

print(X2_train.shape)
print(X2_test.shape)

(375, 2)
(125, 2)


In [288]:
model3 = ScratchSVMClassifier(num_iter=100, 
                              lr= 0.01,
                              kernel='linear', 
                              threshold=1e-5, 
                              verbose=False)

In [289]:
model3.fit(X2_train, y_train)

ValueError: operands could not be broadcast together with shapes (75,75) (375,375) 

In [105]:
from sklearn.model_selection import train_test_split

X = df.iloc[:, [2, 3]].values
y = df.iloc[:, 4].values

X_train, X_test, y_train, y_test =  train_test_split(X, y, test_size=0.25)
print(X_train.shape)
print(X_test.shape)

(75, 2)
(25, 2)


In [253]:
# Irisデータでテスト
model2 = ScratchSVMClassifier(num_iter=50, 
                              lr= 0.01,
                              kernel='linear', 
                              threshold=1e-5, 
                              verbose=False)

In [73]:
from sklearn.datasets import load_iris

#データのダウンロードとXとY変数に代入
iris = load_iris()
df_X = pd.DataFrame(iris.data, columns=iris.feature_names)
df_X.columns = ["sepal_length", "sepal_width", "petal_length", "petal_width"]

df_Y = pd.DataFrame(iris.target)
df_Y.columns = ["Species"]

# データ結合 concat（axis=1）で列に配合
df_0 = pd.concat((df_X, df_Y), axis=1)

# virgicolorとvirginica sepal_lengthとpetal_lengthに再構築する
df = df_0.iloc[50:, :].reset_index()
df = df.drop("index", axis=1)
display(df.head())
display(df.tail())

# 前処理：目的変数【1,2】を【0,1】に置換する
df = df.replace({'Species': {1: 0}})
display(df.head())

df = df.replace({'Species': {2: 1}})
display(df.tail())

,sepal_length,sepal_width,petal_length,petal_width,Species
0,7.0,3.2,4.7,1.4,1
1,6.4,3.2,4.5,1.5,1
2,6.9,3.1,4.9,1.5,1
3,5.5,2.3,4.0,1.3,1
4,6.5,2.8,4.6,1.5,1


,sepal_length,sepal_width,petal_length,petal_width,Species
95,6.7,3.0,5.2,2.3,2
96,6.3,2.5,5.0,1.9,2
97,6.5,3.0,5.2,2.0,2
98,6.2,3.4,5.4,2.3,2
99,5.9,3.0,5.1,1.8,2


,sepal_length,sepal_width,petal_length,petal_width,Species
0,7.0,3.2,4.7,1.4,0
1,6.4,3.2,4.5,1.5,0
2,6.9,3.1,4.9,1.5,0
3,5.5,2.3,4.0,1.3,0
4,6.5,2.8,4.6,1.5,0


,sepal_length,sepal_width,petal_length,petal_width,Species
95,6.7,3.0,5.2,2.3,1
96,6.3,2.5,5.0,1.9,1
97,6.5,3.0,5.2,2.0,1
98,6.2,3.4,5.4,2.3,1
99,5.9,3.0,5.1,1.8,1


In [254]:
model2.fit(X_train, y_train)

置換後 [1.01 0.   1.01 0.   1.01 1.01 1.01 0.   0.   1.01 0.   0.   1.01 0.
 0.   0.   1.01 1.01 1.01 0.   1.01 0.   1.01 0.   1.01 1.01 1.01 0.
 0.   1.01 0.   0.   0.   1.01 0.   0.   0.   0.   1.01 0.   0.   0.
 0.   0.   0.   0.   1.01 1.01 1.01 1.01 1.01 0.   1.01 1.01 0.   1.01
 1.01 0.   1.01 0.   0.   0.   1.01 0.   1.01 0.   0.   1.01 1.01 1.01
 0.   1.01 1.01 1.01 1.01]
[iter0]サポートベクターの数:37
置換後 [1.02 0.   1.02 0.   1.02 1.02 1.02 0.   0.   1.02 0.   0.   1.02 0.
 0.   0.   1.02 1.02 1.02 0.   1.02 0.   1.02 0.   1.02 1.02 1.02 0.
 0.   1.02 0.   0.   0.   1.02 0.   0.   0.   0.   1.02 0.   0.   0.
 0.   0.   0.   0.   1.02 1.02 1.02 1.02 1.02 0.   1.02 1.02 0.   1.02
 1.02 0.   1.02 0.   0.   0.   1.02 0.   1.02 0.   0.   1.02 1.02 1.02
 0.   1.02 1.02 1.02 1.02]
[iter1]サポートベクターの数:37
置換後 [1.03 0.   1.03 0.   1.03 1.03 1.03 0.   0.   1.03 0.   0.   1.03 0.
 0.   0.   1.03 1.03 1.03 0.   1.03 0.   1.03 0.   1.03 1.03 1.03 0.
 0.   1.03 0.   0.   0.   1.03 0.   0.   0.   0.   1.03 

# おかしい挙動とみるべきか。。。
- ラムダiとラムダjの値をランダムに変換する

## 4.SVMとはどのような仕組みか

スクラッチ実装に必要な情報は以上ですが、大まかな仕組みの解説を行います。


SVMは決定境界と近くの点の距離（マージン）を最大化する方法です。特徴量が2つであれば以下の図のように線を引くことを考えます。


https://diveintocode.gyazo.com/a5122026dd2f6c3065ad30bc703e1dbb


決定境界は線形であれば、線形回帰やロジスティック回帰と同様に次の式です。


y
(
x
)
=
w
T
x

$x$ : 特徴量ベクトル


$w$ : 重みベクトル


決定境界とある点 $x$ との距離 $r$ は以下の式で求められます。高校数学で学ぶ「点と直線の距離の公式」や「点と平面の距離の公式」を一般化したものです。


r
=
y
i
y
(
x
)
|
|
w
|
|
=
y
i
w
T
x
|
|
w
|
|

$||w||$ はベクトル $w$ の大きさで、特徴量が2つならば $||w|| = \sqrt{w_{1}^2+w_{2}^2}$ です。また、学習するi番目のデータのラベルを $y_{i}=-1$または$y_{i}=1$ としています。


訓練データの中で「最も距離 $r$ が短くなる点x（サポートベクター）の距離 $r$ を最大化する $w$ を求める」ことがSVMによる分類だと言い換えられます。


これは非常に複雑な問題です。 $w$ を変化させると、最も距離 $r$ が短くなる点x（サポートベクター）も変化していくからです。計算するためには、問題を扱いやすい形に変形させる必要があります。


扱いやすい形にする
まず定数 $M(>0)$ を置くと、先ほどの問題は、


「$\frac{M}{||w||}$ を $y_{i}(w^{T}x_{i})\geq M$ という条件の元で最大化する $w$ や $M$ を求める問題」


と表現できます。条件式は $x_{i}$ に訓練データの全ての点を入れて成り立つ必要があります。まだまだややこしいですが、これをMで割ってしまいます。そうすると、


「$\frac{1}{||w||}$ を $y_{i}(\frac{w^{T}}{M}x_{i})\geq 1$ という条件の元で最大化する $w$ や $M$ を求める問題」


になり、さらに $w^{T} \gets \frac{w^{T}}{M}$ と置き換えてしまいます。


そうすれば、


「 $\frac{1}{||w||}$ を $y_{i}(w^{T}X_{i})\geq 1$ という条件の元で最大化する $w$を求める問題」


まで簡単化できます。 $\frac{1}{||w||}$ を最大化するというのは $||w||$ を最小化することと同じです。これを後々さらに扱いやすくするために $\frac{1}{2}||w||^2$ を最小化すると考えます。よって、


「 $\frac{1}{2}||w||^2$ を $y_{i}(w^{T}x_{i})\geq 1$ という条件の元で最小化する $w$ を求める問題」


とすることができます。


解きやすい問題にする（双対化）
こういった不等式制約を持つ最適化問題は次のように ラグランジュの未定乗数法 で置き換えられることが知られています。


なお、このように難しい問題を別の簡単な問題に言い換えることを 双対化する といいます。


ラグランジュの未定乗数法を用いると以下のラグランジュ関数が得られます。


L
(
w
,
λ
)
=
1
2
|
|
w
|
|
2
−
N
∑
i
=
1
 
λ
i
{
y
i
(
w
T
x
i
)
−
1
}

$\lambda$ はラグランジュ乗数と呼ばれる数で、0以上の値です。これを $w$について微分し、0に等しいと置くと、次の式が得られます。


w
=
N
∑
i
=
1
 
λ
i
y
i
x
i

（この微分のために $||w||$ ではなく $\frac{1}{2}||w||^2$ としています）


これをラグランジュ関数に代入して整理すると


N
∑
i
=
1
 
λ
i
−
1
2
N
∑
i
=
1
  
N
∑
j
=
1
 
λ
i
λ
j
ｙ
i
ｙ
j
x
T
i
x
j

を $\lambda_{i} \geq 0$ かつ $\sum_{n=1}^{N}\lambda_{i}y_{i} = 0$ の条件の元で最大化するときの $\lambda_{i}$ を探す問題に双対化できます。


この形になれば、$\lambda$ を勾配降下法により求めることができます。$w$は出てきませんが、得られる結果は同じです。


カーネル
最後の式の $x_{i}^{T} x_j$ の部分を $k(x_i, x_j)$ という関数に置き換えます。この関数を カーネル関数 と呼びます。


N
∑
i
=
1
 
λ
i
−
1
2
N
∑
i
=
1
  
N
∑
j
=
1
 
λ
i
λ
j
ｙ
i
ｙ
j
k
(
x
i
,
x
j
)

この式が問題1の最急降下法の式の元になります。


カーネル関数は $x_{i}^{T} x_j$ ではない様々な計算に置き換えることができます。この部分を置き換えるだけで、元の特徴量を 高次元空間 に移動させたことと同じ結果が得られ、高い分類性能を得ることができます。これを カーネルトリック と呼びます。


高次元へ移す簡単な例
次の図のように1次元上に2色の点があるとします。これらを直線一本を引くことで分けることは不可能です。


https://diveintocode.gyazo.com/e75b3c8d2692afa9fba4f76485a883eb


しかし、例えば以下のように変換してみると直線でも分けられそうです。
$x^2=2.5$ あたりに線を引くことになります。


https://diveintocode.gyazo.com/5f5bacd8aac5831f1936759362b8a184


これは$x^2$を計算し、それを縦軸にプロットしたグラフです。1次元だったデータを $\phi(x)=x^2$ の関数により高次元（2次元）へと移動しました。


こういったことをSVMはカーネルトリックにより行います。